In [11]:
import pandas as pd
import os
from tqdm import tqdm
from utils import avg
from utils import evidence_to_mask

def to_data_df_movies(df, data_dir):
    data_df = []
    columns = ['text', 'classification', 'rationale']
    for i in tqdm(range(len(df))):
        df_row = df.loc[i]
        
        doc_id = df_row['annotation_id']
        evidence_list = df_row['evidences']
        if evidence_list:
            evidence_list = evidence_list[0]
        classification = df_row['classification']
        
        text = ''
        file = f'{data_dir}/docs/{doc_id}'
        if os.path.isfile(file):
            f = open(file, 'r', encoding="utf-8") 
            for line in f.readlines():
                text += line.rstrip() + ' '
        else:
            print("???")
            print(file)
            quit()
        
        tokens = text.split()
        rationale_mask = evidence_to_mask(tokens, evidence_list)
        
        # joining text and query with [SEP]
#         QA = f"[CLS] {text} [SEP]" ???
        QA = f"{text}"
        rationale_mask = rationale_mask
    
        data_df.append([QA, classification, rationale_mask])
    data_df = pd.DataFrame(data_df, columns=columns)
    return data_df

In [4]:
dataset = "movies"

In [5]:
data_dir = f'../data/{dataset}'
train = pd.read_json(f'{data_dir}/train.jsonl', lines=True)
test = pd.read_json(f'{data_dir}/test.jsonl', lines=True)
val = pd.read_json(f'{data_dir}/val.jsonl', lines=True)

In [6]:
train.info()
# test.info()
# val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   annotation_id   1600 non-null   object 
 1   classification  1600 non-null   object 
 2   evidences       1600 non-null   object 
 3   query           1600 non-null   object 
 4   query_type      0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 62.6+ KB


In [12]:
train_data_df = to_data_df_movies(train, data_dir)
train_data_df.to_csv(f"{dataset}/train.csv",index_label="id")
test_data_df = to_data_df_movies(test, data_dir)
test_data_df.to_csv(f"{dataset}/test.csv",index_label="id")
val_data_df = to_data_df_movies(val, data_dir)
val_data_df.to_csv(f"{dataset}/val.csv",index_label="id")

 60%|██████████████████████████████████████████████▏                              | 961/1600 [00:00<00:00, 1005.00it/s]


IndexError: list index out of range